In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [5]:
# Parse the html result with soup 
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Retrieve the latest title
news_title = soup.select_one('div.content_title a').text

In [7]:
# Retrieve the latest paragraph
news_p = soup.select_one('div.article_teaser_body').text

In [8]:
# The latest News Title and Paragraph Text
print(f'The latest news title: {news_title} \nParagraph text: {news_p}')

The latest news title: Hear Audio From NASA's Perseverance As It Travels Through Deep Space 
Paragraph text: The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.


## JPL Mars Space Images - Featured Image

In [9]:
# Visit URL
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [10]:
# Find the full image botton and click
img_elem = browser.find_by_id('full_image')
img_elem.click()

In [11]:
# Find the more info button and click
browser.is_element_present_by_text('more info', wait_time=3)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [12]:
# Parse the html result with soup
img_html = browser.html
img_soup = bs(img_html, 'html.parser')

In [13]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA16105_hires.jpg'

In [14]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16105_hires.jpg'

## Mars Facts

In [15]:
# URL
facts_url = 'https://space-facts.com/mars/'

# Read html by Pandas
tables = pd.read_html(facts_url)

# Slice off dataframes that needed
mars_df = tables[0]

# Set columns
mars_df.columns = ['Description', 'Mars']
mars_df.set_index('Description', inplace=True)

mars_html = mars_df.to_html()

mars_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

## Mars Hemispheres

In [16]:
hem_list = ['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']
hemisphere_image_urls = []

for hem in hem_list:
    # Visit URL
    hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(hem_url)
    
    # Find the hemisphere images and click
    hem_elem = browser.links.find_by_partial_text(hem)
    hem_elem.click()
    
    # Find original to download full img
    hem_elem_org = browser.find_by_text("Original", wait_time=1)
    hem_elem_org.click()
    
    # Scrape the Mars website 
    hem_html = browser.html
    hem_soup = bs(hem_html, 'html.parser')
    
    hem_title = hem_soup.h2.text
    hem_url = hem_soup.select_one('li a')['href']
    
    hem_dict = {
        'title': hem_title,
        'img_url': hem_url
    }
    
    hemisphere_image_urls.append(hem_dict)

In [17]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]